In [1]:
import pandas as pd
from tensorflow.python import keras # High level, easy to use, intuitive DNN library
from tensorflow.python.keras.datasets import mnist # Digits data
from tensorflow.python.keras.models import Sequential, Model # Method that helps us build the model,
                                                      # by stacking the layers together
from tensorflow.python.keras.layers import Dense, Dropout, Flatten # Different layers as part of the network;
# Dense layer - is your regular linear layer, # Dropt out sets some of the weights to zero randomly
# to prevent from overfitting; Flatten does exactly what it sounds like, flattens the multi-dimensional array
# into one dimension
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D #2D convolution array, and Max Pooling array
from tensorflow.python.keras import backend as K
import numpy as np
import gc
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
obs = pd.read_csv('C:/Users/Max/Downloads/data/training_set.csv')
meta = pd.read_csv('C:/Users/Max/Downloads/data/training_set_metadata.csv')

In [3]:
obs2=obs[['object_id','flux']]

In [44]:
max(obs.groupby('object_id').apply(len))

352

In [4]:
from time import time
t0=time()
res = obs2.assign(ValNum=obs2.groupby('object_id').cumcount()+1)\
        .pivot(index='object_id', columns='ValNum', values='flux')\
        .reset_index()
t1=time()
print(t1-t0)

0.7187788486480713


In [48]:
from time import time
t0=time()
res2 = obs.assign(ValNum=obs2.groupby('object_id').cumcount()+1)\
        .pivot(index='object_id', columns='ValNum', values='detected')\
        .reset_index()
t1=time()
print(t1-t0)

0.8437988758087158


In [5]:
res=res.fillna(0)

In [31]:
train_x=res.loc[:7000,:].drop(['object_id'],axis=1).values
test_x=res.loc[7001:,:].drop(['object_id'],axis=1).values
train_y=pd.get_dummies(meta.loc[:7000,:]['target']).values
test_y=pd.get_dummies(meta.loc[7001:,:]['target']).values

In [7]:
train_x.shape

(7001, 352)

In [52]:
np.dstack((train_x,train_x)).shape

(7001, 352, 2)

In [32]:
train_x = train_x.reshape(7001,352,1)
test_x = test_x.reshape(847,352,1)


In [47]:
from keras.models import Model
from keras.layers import Conv1D, Dense, MaxPool1D, Flatten, Input
import numpy as np
seq_length = 352
inp =  Input(shape=(352, 1))

conv = Conv1D(filters=2, kernel_size=2)(inp)
pool = MaxPool1D(pool_size=2)(conv)
flat = Flatten()(pool)
dense = Dense(14)(flat)
model = Model(inp, dense)
#model.compile(loss='mse', optimizer='adam')

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [37]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 352, 1)            0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 351, 2)            6         
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 175, 2)            0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 350)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 14)                4914      
Total params: 4,920
Trainable params: 4,920
Non-trainable params: 0
_________________________________________________________________
None


In [38]:
model.fit(train_x, train_y, batch_size=16, epochs=10)
score = model.evaluate(test_x, test_y, batch_size=16)

Epoch 1/10
7001/7001 [==============================] - 2s 308us/step - loss: 8.4533 - acc: 0.0863
Epoch 2/10
7001/7001 [==============================] - 1s 163us/step - loss: 8.0687 - acc: 0.1770
Epoch 3/10
7001/7001 [==============================] - 1s 183us/step - loss: 8.2329 - acc: 0.1835
Epoch 4/10
7001/7001 [==============================] - 1s 183us/step - loss: 8.2268 - acc: 0.1884
Epoch 5/10
7001/7001 [==============================] - 1s 163us/step - loss: 8.0908 - acc: 0.2005
Epoch 6/10
7001/7001 [==============================] - 1s 163us/step - loss: 8.4540 - acc: 0.2310
Epoch 7/10
7001/7001 [==============================] - 1s 185us/step - loss: 8.2946 - acc: 0.2298
Epoch 8/10
7001/7001 [==============================] - 1s 163us/step - loss: 8.3712 - acc: 0.2065
Epoch 9/10
7001/7001 [==============================] - 1s 161us/step - loss: 8.3353 - acc: 0.1993
Epoch 10/10
847/847 [==============================] - 0s 184us/step


In [28]:

from keras.models import Model
from keras.layers import Conv1D, Dense, MaxPool1D, Flatten, Input
import numpy as np

inp =  Input(shape=(352, 1))
conv = Conv1D(filters=2, kernel_size=2)(inp)
pool = MaxPool1D(pool_size=2)(conv)
flat = Flatten()(pool)
dense = Dense(1)(flat)
model = Model(inp, dense)
model.compile(loss='mse', optimizer='adam')

print(model.summary())

# get some data
X = np.expand_dims(np.random.randn(10, 352), axis=2)
y = np.random.randn(10, 1)

# fit model
model.fit(X, y)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 352, 1)            0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 351, 2)            6         
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 175, 2)            0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 350)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 351       
Total params: 357
Trainable params: 357
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1
10/10 [==============================] - 0s 28ms/step - loss: 1.3270


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(seq_length, 5)))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=16, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=16)